In [1]:
import pandas as pd

from sklearn.preprocessing import StandardScaler

from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
#import data
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [3]:
train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [4]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
#survived- strongly depends on Pclass

train[['Pclass', 'Survived']].groupby(['Pclass']).mean()

,Survived
Pclass,
1,0.629630
2,0.472826
3,0.242363


In [6]:
#survived-74% of F survived and only 19% M 

train[['Sex', 'Survived']].groupby(['Sex']).mean()

,Survived
Sex,
female,0.742038
male,0.188908


In [7]:
train[['SibSp', 'Survived']].groupby(['SibSp']).mean()

,Survived
SibSp,
0,0.345395
1,0.535885
2,0.464286
3,0.250000
4,0.166667
5,0.000000
8,0.000000


In [8]:
train[['Parch', 'Survived']].groupby(['Parch']).mean()

,Survived
Parch,
0,0.343658
1,0.550847
2,0.500000
3,0.600000
4,0.000000
5,0.200000
6,0.000000


In [9]:
#parch and sibsp can be combined in one feature
train['Family']=train['SibSp']+train['Parch']
#train.loc[train['Family']==0,'Age'] = 0
#train.loc[(train['Family']>0) & (train['Family'] <= 3),'Family'] = 1
#train.loc[train['Family']>3,'Family'] = 4


test['Family']=test['SibSp']+test['Parch']
#test.loc[test['Family']==0,'Age'] = 0
#test.loc[(test['Family']>0) & (test['Family'] <= 3),'Family'] = 1
#test.loc[test['Family']>3,'Family'] = 4

train[['Family', 'Survived']].groupby(['Family']).mean()

,Survived
Family,
0,0.303538
1,0.552795
2,0.578431
3,0.724138
4,0.200000
5,0.136364
6,0.333333
7,0.000000
10,0.000000


In [10]:
#drop Ticket and cabin(info on cabin missing in most of the cases)
train=train.drop(['Ticket','Cabin'], axis = 1)
test=test.drop(['Ticket','Cabin'], axis = 1)

In [11]:
#extract info oabout the title
train['Title'] = train.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
pd.crosstab(train['Title'], train['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


In [12]:
test['Title'] = test.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
pd.crosstab(test['Title'], test['Sex'])

Sex,female,male
Title,,
Col,0,2
Dona,1,0
Dr,0,1
Master,0,21
Miss,78,0
Mr,0,240
Mrs,72,0
Ms,1,0
Rev,0,2


In [13]:
#group title: miss=1,mrs=2,mr=3,mater=4,the rest=5
#most of mrs and miss survived, mr only 16% survived
train['Title'] = train['Title'].replace(['Miss','Mlle'],1)
train['Title'] = train['Title'].replace(['Mrs','Ms','Mme'],2)
train['Title'] = train['Title'].replace('Mr',3)
train['Title'] = train['Title'].replace('Master',4)
train['Title'] = train['Title'].replace(['Capt', 'Col', 'Dr','Major','Rev'],5)
train['Title'] = train['Title'].replace(['Countess','Don','Dona','Jonkheer', 'Lady', 'Sir'], 6)
train['Title'] = train['Title'].fillna(0)    
train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,1,0.701087
1,2,0.795276
2,3,0.156673
3,4,0.575000
4,5,0.277778
5,6,0.600000


In [14]:
test['Title'] = test['Title'].replace(['Miss','Mlle'],1)
test['Title'] = test['Title'].replace(['Mrs','Ms','Mme'],2)
test['Title'] = test['Title'].replace('Mr',3)
test['Title'] = test['Title'].replace('Master',4)
test['Title'] = test['Title'].replace(['Capt', 'Col', 'Dr','Major','Rev'],5)
test['Title'] = test['Title'].replace(['Countess','Don','Dona','Jonkheer', 'Lady', 'Sir'], 6)
test['Title'] = test['Title'].fillna(0)    

In [15]:
#get rid of name, parch and sibsp
train = train.drop(['Name','SibSp','Parch'], axis=1)
test = test.drop(['Name','SibSp','Parch'], axis=1)

In [16]:
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [17]:
#rename female=0, male=1
train['Sex'] = train['Sex'].replace("female",0)
train['Sex'] = train['Sex'].replace("male",1)
train['Sex'] = train['Sex'].fillna(test["Sex"].mode())

test['Sex'] = test['Sex'].replace("female",0)
test['Sex'] = test['Sex'].replace("male",1)
test['Sex'] = test['Sex'].fillna(test["Sex"].mode())

#rename S=0, Q=1, C=2
train['Embarked'] = train['Embarked'].replace("S",0)
train['Embarked'] = train['Embarked'].replace("Q",1)
train['Embarked'] = train['Embarked'].replace("C",2)
train['Embarked'] = train['Embarked'].fillna(0)

test['Embarked'] = test['Embarked'].replace("S",0)
test['Embarked'] = test['Embarked'].replace("Q",1)
test['Embarked'] = test['Embarked'].replace("C",2)
test['Embarked'] = test['Embarked'].fillna(0)


In [18]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Family,Title
0,1,0,3,1,22.0,7.2500,0.0,1,3
1,2,1,1,0,38.0,71.2833,2.0,1,2
2,3,1,3,0,26.0,7.9250,0.0,0,1
3,4,1,1,0,35.0,53.1000,0.0,1,2
4,5,0,3,1,35.0,8.0500,0.0,0,3


In [19]:
#fillna age
train['Age'].fillna(train.groupby('Title')['Age'].transform("median"), inplace= True)
test['Age'].fillna(test.groupby('Title')['Age'].transform("median"), inplace= True)


In [20]:
#fillna fare
train['Fare'].fillna(train.groupby('Title')['Fare'].transform("median"), inplace= True)
test['Fare'].fillna(test.groupby('Title')['Fare'].transform("median"), inplace= True)


In [21]:
#form age groups
train.loc[train['Age']<=16,'Age'] = 0
train.loc[(train['Age']>16) & (train['Age'] <= 26),'Age'] = 1
train.loc[(train['Age']>26) & (train['Age'] <= 36),'Age'] = 2
train.loc[(train['Age']>36) & (train['Age'] <= 62),'Age'] = 3
train.loc[train['Age']>62,'Age'] = 4
train.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Family,Title
0,1,0,3,1,1.0,7.2500,0.0,1,3
1,2,1,1,0,3.0,71.2833,2.0,1,2
2,3,1,3,0,1.0,7.9250,0.0,0,1
3,4,1,1,0,2.0,53.1000,0.0,1,2
4,5,0,3,1,2.0,8.0500,0.0,0,3


In [22]:
#form age groups
test.loc[test['Age']<=16,'Age'] = 0
test.loc[(test['Age']>16) & (test['Age'] <= 26),'Age'] = 1
test.loc[(test['Age']>26) & (test['Age'] <= 36),'Age'] = 2
test.loc[(test['Age']>36) & (test['Age'] <= 62),'Age'] = 3
test.loc[test['Age']>62,'Age'] = 4
test.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked,Family,Title
0,892,3,1,2.0,7.8292,1,0,3
1,893,3,0,3.0,7.0000,0,1,2
2,894,2,1,3.0,9.6875,1,0,3
3,895,3,1,2.0,8.6625,0,0,3
4,896,3,0,1.0,12.2875,0,2,2


In [23]:
#form fare groups
train.loc[train['Fare']<=10.0,'Fare'] = 0
train.loc[(train['Fare']>10.0) & (train['Fare'] <= 20.0),'Fare'] = 1
train.loc[(train['Fare']>20.0) & (train['Fare'] <= 30.0),'Fare'] = 2
train.loc[(train['Fare']>30.0) & (train['Fare'] <= 50.0),'Fare'] = 3
train.loc[train['Fare']>50.0,'Fare'] = 4


In [24]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Family,Title
0,1,0,3,1,1.0,0.0,0.0,1,3
1,2,1,1,0,3.0,4.0,2.0,1,2
2,3,1,3,0,1.0,0.0,0.0,0,1
3,4,1,1,0,2.0,4.0,0.0,1,2
4,5,0,3,1,2.0,0.0,0.0,0,3


In [25]:
train.shape

(891, 9)

In [26]:

train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
Fare           0
Embarked       0
Family         0
Title          0
dtype: int64

In [27]:
#form fare groups
test.loc[test['Fare']<=10.0,'Fare'] = 0
test.loc[(test['Fare']>10.0) & (test['Fare'] <= 20.0),'Fare'] = 1
test.loc[(test['Fare']>20.0) & (test['Fare'] <= 30.0),'Fare'] = 2
test.loc[(test['Fare']>30.0) & (test['Fare'] <= 50.0),'Fare'] = 3
test.loc[test['Fare']>50.0,'Fare'] = 4
test.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked,Family,Title
0,892,3,1,2.0,0.0,1,0,3
1,893,3,0,3.0,0.0,0,1,2
2,894,2,1,3.0,0.0,1,0,3
3,895,3,1,2.0,0.0,0,0,3
4,896,3,0,1.0,1.0,0,2,2


In [28]:
test.shape

(418, 8)

In [29]:
X_train=train.drop(['Survived','PassengerId'], axis = 1)
Y_train = train['Survived']

X_test=test.drop(['PassengerId'], axis = 1)

In [30]:
# Naive Bayes classifiers
gauss = GaussianNB()
gauss.fit(X_train, Y_train)
Y_pred = gauss.predict(X_test)
acc1 = gauss.score(X_train, Y_train)
acc1

0.77890011223344557

In [31]:
# LogisticRegression
logreg = LogisticRegression(C=0.5, solver='newton-cg', multi_class='multinomial',max_iter=10000)
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc2 = logreg.score(X_train, Y_train) 
acc2

0.80920314253647585

In [32]:
#perceptron
perceptron = Perceptron(alpha=0.00001,max_iter=800)
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc3 = perceptron.score(X_train, Y_train) 
acc3

0.78787878787878785

In [33]:
# Random Forest Classifier
random_forest = RandomForestClassifier(n_estimators=50, criterion='entropy', min_samples_split=2, min_samples_leaf=1)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc4 = random_forest.score(X_train, Y_train) 
acc4

0.8843995510662177

In [34]:
# SGD Classifier
sgd = SGDClassifier(loss='log', penalty='l2', alpha=0.0001, l1_ratio=0.25,max_iter=800)
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc5 = sgd.score(X_train, Y_train) 
acc5

0.80359147025813693

In [35]:
# Decision Tree Classifier
decision_tree = DecisionTreeClassifier(criterion='entropy', splitter='random', min_samples_split=2, min_samples_leaf=1)
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc6 = decision_tree.score(X_train, Y_train) 
acc6

0.8843995510662177

In [36]:
# LogisticRegression
logreg = LogisticRegression(C=0.5, solver='newton-cg', multi_class='multinomial',max_iter=10000)
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc7 = logreg.score(X_train, Y_train) 
acc7

0.80920314253647585

In [37]:
# Random Forest Classifier
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc8 = random_forest.score(X_train, Y_train) 
acc8

0.8843995510662177

In [38]:
# K Nearest Neighbors classifiers
knn = KNeighborsClassifier(algorithm='auto',  metric='minkowski', 
                           metric_params=None, n_jobs=1, n_neighbors=3, p=2, 
                           weights='uniform')
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc9 = knn.score(X_train, Y_train)
acc9

0.85746352413019078

In [39]:
# SVC classifiers
svc = SVC(kernel='poly')
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc10 = svc.score(X_train, Y_train) 
acc10

0.85072951739618408

In [40]:
submit = pd.DataFrame({
        "PassengerId": test['PassengerId'],
        "Survived": Y_pred
    })
submit.to_csv('./submission.csv', index=False)